<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "input"
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

### Download Data

We will use [Walmart annual report PDFs](https://github.com/sujee/data/tree/main/data-prep-kit/walmart-reports-1) as our input data.

Feel free to substitute your data

In [2]:
import os, sys
import shutil
from utils import download_file

## Download the data files
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart-10K-Reports-Optimized_2023.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart-10K-Reports-Optimized_2023.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024_copy.pdf' ))  # create a dupe file

Local file 'input/Walmart-10K-Reports-Optimized_2023.pdf' (1.61 MB) already exists. Skipping download.
Local file 'input/Walmart_2024.pdf' (4.87 MB) already exists. Skipping download.
Local file 'input/Walmart_2024_copy.pdf' (4.87 MB) already exists. Skipping download.


### Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_exact_dedupe_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_docid_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_doc_quality_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_doc_quality_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '07_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### Import Common python modules

In [4]:
import os
import sys

# Main repo root
from utils import rootdir

from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

STAGE = 0

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### Set Input/output Folder

In [5]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'


### Execute 

In [6]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


22:47:19 INFO - Running locally
22:47:19 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
22:47:19 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
22:47:19 INFO - data factory data_ max_files -1, n_sample -1
22:47:19 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
22:47:19 INFO - pipeline id pipeline_id
22:47:19 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
22:47:19 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
22:47:19 INFO - actor creation delay 0
22:47:19 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
202

✅ Stage:1 completed successfully
CPU times: user 4.59 s, sys: 1.17 s, total: 5.76 s
Wall time: 12min 27s


### Inspect Generated output

Here we should see one entry per input file processed

In [7]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (3, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,Walmart-10K-Reports-Optimized_2023.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,Walmart-10K-Reports-Optimized_2023.pdf
1,Walmart_2024_copy.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,83,1058,835fe3a8-999a-49bc-84f9-ab34eee6f7d6,pdf,f9a875d8dfe00abad97c66a0be7d501f01ad8173b3da33...,1215215,2024-09-15T22:59:31.870327,352.081304,Walmart_2024_copy.pdf
2,Walmart_2024.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,83,1058,0b23e209-8976-4158-a6cd-a3541495421f,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-15T22:53:39.613034,366.699574,Walmart_2024.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### Set Input/output Folder

In [8]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### Execute 

In [9]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

22:59:44 INFO - Running locally
22:59:44 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
22:59:44 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
22:59:44 INFO - data factory data_ max_files -1, n_sample -1
22:59:44 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
22:59:44 INFO - pipeline id pipeline_id
22:59:44 INFO - code location None
22:59:44 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
22:59:44 INFO - actor creation delay 0
22:59:44 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type': 'r

✅ Stage:2 completed successfully
CPU times: user 955 ms, sys: 302 ms, total: 1.26 s
Wall time: 19 s


### Inspect Generated output

We would see documents are split into many chunks

In [10]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 3
Chunks created : 1,973
Input data dimensions (rows x columns)=  (3, 12)
Output data dimensions (rows x columns)=  (1973, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
96,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,Walmart-10K-Reports-Optimized_2023.pdf,Strategic Risks\nA failure to respond effectiv...,$.main-text[202],18,"[46.3991394, 604.15112305, 538.12310791, 650.7..."
513,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,Walmart-10K-Reports-Optimized_2023.pdf,Other Opioid Related Litigation\nDerivative ac...,$.main-text[881],78,"[46.49466324, 35.41732788, 540.34527588, 105.8..."
413,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,Walmart-10K-Reports-Optimized_2023.pdf,Advertising Costs\nAdvertising costs are expen...,$.main-text[710],66,"[46.37985611, 281.76379395, 539.89404297, 316...."


## Step-4: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### Set Input/output Folder

In [11]:
STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_exact_dedupe_out'


### Execute 

In [12]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:00:02 INFO - Running locally
23:00:02 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
23:00:02 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_exact_dedupe_out
23:00:02 INFO - data factory data_ max_files -1, n_sample -1
23:00:02 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:00:02 INFO - pipeline id pipeline_id
23:00:02 INFO - code location None
23:00:02 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:00:02 INFO - actor creation delay 0
23:00:02 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
2024-09-15 23:00:04,285	INFO worker.py:1744 -- Started a local Ray instance. View the das

✅ Stage:3 completed successfully
CPU times: user 103 ms, sys: 156 ms, total: 259 ms
Wall time: 15 s


### Inspect Generated output

In [13]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (1973, 15)
Output data dimensions (rows x columns)=  (2, 16)
Input chunks before exact dedupe : 1,973
Output chunks after exact dedupe : 2
Duplicate chunks removed :   1971


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,removed
1,Walmart_2024.pdf,100,83,1058,0b23e209-8976-4158-a6cd-a3541495421f,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-15T22:53:39.613034,366.699574,Walmart_2024.pdf,"A message from our CEO\n""At Walmart, we're a p...",$.main-text[6],2,"[30.18040848, 338.63272095, 526.46081543, 436....","[0b23e209-8976-4158-a6cd-a3541495421f, 0b23e20..."
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,Walmart-10K-Reports-Optimized_2023.pdf,"A message from our CEO\n""We started this new y...",$.main-text[6],2,"[29.09381294, 166.47808838, 500.76074219, 270....","[812f70a8-bb7a-4e30-8153-a48b00a1cc73, 812f70a..."


## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

In [14]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  = 4

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_exact_dedupe_out' --> output='output/04_docid_out'


In [15]:
%%time 

from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:00:17 INFO - Running locally
23:00:17 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column', 'start_id': 0}
23:00:17 INFO - data factory data_ is using local data access: input_folder - output/03_exact_dedupe_out output_folder - output/04_docid_out
23:00:17 INFO - data factory data_ max_files -1, n_sample -1
23:00:17 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:00:17 INFO - pipeline id pipeline_id
23:00:17 INFO - code location None
23:00:17 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:00:17 INFO - actor creation delay 0
23:00:17 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
2024-09-15 23:00:19,391	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orches

✅ Stage:4 completed successfully
CPU times: user 129 ms, sys: 154 ms, total: 283 ms
Wall time: 15 s


### Inspect Generated output

In [16]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 16)
Output data dimensions (rows x columns)=  (2, 18)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,removed,hash_column,int_id_column
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,Walmart-10K-Reports-Optimized_2023.pdf,"A message from our CEO\n""We started this new y...",$.main-text[6],2,"[29.09381294, 166.47808838, 500.76074219, 270....","[812f70a8-bb7a-4e30-8153-a48b00a1cc73, 812f70a...",a651a5dc74053a1d5c92d134baf9ea23b7c009036a2619...,0
1,Walmart_2024.pdf,100,83,1058,0b23e209-8976-4158-a6cd-a3541495421f,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-15T22:53:39.613034,366.699574,Walmart_2024.pdf,"A message from our CEO\n""At Walmart, we're a p...",$.main-text[6],2,"[30.18040848, 338.63272095, 526.46081543, 436....","[0b23e209-8976-4158-a6cd-a3541495421f, 0b23e20...",01b953276ba54dfa57da3f73dfcbbd26e62b634e8a3a41...,1


## Step-6: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### Set Input/output Folder

In [17]:
## Input to this component is the output of doc_id generator component. 

STAGE  = 5

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_docid_out' --> output='output/05_fuzzy_dedupe_out'


### Execute 

In [18]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.5,
    "fdedup_doc_cpu": 0.5,
    "fdedup_mhash_cpu": 0.5,
    "fdedup_num_doc_actors": 2,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 2,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:00:32 INFO - Running locally
23:00:32 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.5, 'mhash_cpu': 0.5, 'doc_cpu': 0.5, 'num_doc_actors': 2, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 2, 'num_permutations': 64, 'threshold': 0.8, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
23:00:32 INFO - data factory data_ is using local data access: input_folder - output/04_docid_out output_folder - output/05_fuzzy_dedupe_out
23:00:32 INFO - data factory data_ max_files -1, n_sample -1
23:00:32 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:00:32 INFO - pipeline id pipeline_id
23:00:32 INFO - code location None
23:00:32 INFO - number of

✅ Stage:5 completed successfully
CPU times: user 172 ms, sys: 164 ms, total: 336 ms
Wall time: 32.1 s


### Inspect Generated output

In [19]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 16)
Output data dimensions (rows x columns)=  (2, 18)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,removed,int_id_column,hash_column
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,Walmart-10K-Reports-Optimized_2023.pdf,"A message from our CEO\n""We started this new y...",$.main-text[6],2,"[29.09381294, 166.47808838, 500.76074219, 270....","[812f70a8-bb7a-4e30-8153-a48b00a1cc73, 812f70a...",0,-1
1,Walmart_2024.pdf,100,83,1058,0b23e209-8976-4158-a6cd-a3541495421f,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-15T22:53:39.613034,366.699574,Walmart_2024.pdf,"A message from our CEO\n""At Walmart, we're a p...",$.main-text[6],2,"[30.18040848, 338.63272095, 526.46081543, 436....","[0b23e209-8976-4158-a6cd-a3541495421f, 0b23e20...",1,-1


## Step-7: Document Quality

### Set Input/output Folder

In [20]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_doc_quality_dir
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_doc_quality_out'


### Execute 

In [21]:
%%time

import os
import sys
from pathlib import Path

from doc_quality_transform import (
    text_lang_cli_param,
    doc_content_column_cli_param,
    bad_word_filepath_cli_param,
)
from doc_quality_transform_ray import DocQualityRayTransformConfiguration
from data_processing.utils import ParamsUtils

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

doc_quality_basedir = os.path.join(rootdir, "transforms", "language", "doc_quality", "python")
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_creation_delay": 0,
    # doc quality configuration
    text_lang_cli_param: "en",
    doc_content_column_cli_param: "contents",
    bad_word_filepath_cli_param: os.path.join(doc_quality_basedir, "ldnoobw", "en"),
}


Path(output_folder).mkdir(parents=True, exist_ok=True)

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocQualityRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:01:04 INFO - Running locally
23:01:04 INFO - doc_quality parameters are : {'text_lang': 'en', 'doc_content_column': 'contents', 'bad_word_filepath': '/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee-dev/transforms/language/doc_quality/python/ldnoobw/en', 's3_cred': None, 'docq_data_factory': <data_processing.data_access.data_access_factory.DataAccessFactory object at 0x78a6313242d0>}
23:01:04 INFO - data factory docq_ is using local configuration without input/output path
23:01:04 INFO - data factory docq_ max_files -1, n_sample -1
23:01:04 INFO - data factory docq_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:01:04 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_doc_quality_out
23:01:04 INFO - data factory data_ max_files -1, n_sample -1
23:01:04 INFO - data factory data_ Not using data sets, checkpointin

✅ Stage:6 completed successfully
CPU times: user 105 ms, sys: 164 ms, total: 269 ms
Wall time: 15.2 s


### Inspect Generated output

In [22]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 16)
Output data dimensions (rows x columns)=  (2, 29)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_mean_word_len,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words
1,Walmart_2024.pdf,100,83,1058,0b23e209-8976-4158-a6cd-a3541495421f,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-15T22:53:39.613034,366.699574,...,5.16000,0.0,4,0.0,0.0,False,0.0,0.0,1.0,True
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,...,4.83871,0.0,3,0.0,0.0,False,0.0,0.0,1.0,True


## Step-8:   Text encoding

Encode text for the vector storage.

In [23]:
STAGE  = 7

input_folder = output_doc_quality_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-7: Processing input='output/06_doc_quality_out' --> output='output/07_embeddings_out'


In [24]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:01:20 INFO - Running locally
23:01:20 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
23:01:20 INFO - data factory data_ is using local data access: input_folder - output/06_doc_quality_out output_folder - output/07_embeddings_out
23:01:20 INFO - data factory data_ max_files -1, n_sample -1
23:01:20 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:01:20 INFO - pipeline id pipeline_id
23:01:20 INFO - code location None
23:01:20 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:01:20 INFO - actor creation delay 0
23:01:20 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
2024-09-15 23:01:22,555	INFO worker.py:1744 -- Started a local Ray instance. 

✅ Stage:7 completed successfully
CPU times: user 588 ms, sys: 249 ms, total: 837 ms
Wall time: 22.1 s


### Inspect Generated output

In [25]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 29)
Output data dimensions (rows x columns)=  (2, 30)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words,embeddings
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,812f70a8-bb7a-4e30-8153-a48b00a1cc73,pdf,56eb1501e8cc4a4cb452a7457049c3184e994d35e67086...,1255786,2024-09-15T22:54:02.048322,389.136621,...,0.0,3,0.0,0.0,False,0.0,0.0,1.0,True,"[0.014166231, 0.078922085, 0.068165645, -0.018..."
1,Walmart_2024.pdf,100,83,1058,0b23e209-8976-4158-a6cd-a3541495421f,pdf,cd408a97e6d67a9044e6992b6d0c9f553b9522e4b18123...,1215210,2024-09-15T22:53:39.613034,366.699574,...,0.0,4,0.0,0.0,False,0.0,0.0,1.0,True,"[0.0074538216, 0.08018822, 0.051178854, 0.0021..."


## Step-9: Copy output to final output dir

In [26]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/07_embeddings_out' --> 'output/output_final'
